# Gamma Matrix Estimation

In this notebook, I am going to use the pyGSTi data to estimate the state-preparation-error-free response matrix. 

In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
# Validation
rho0 = np.array([[0.9945645, -0.0085467],
                 [-0.0269976, 0.0054355]])

Gx = np.array([[1, 0, 0, 0], 
               [0.0003134, 0.9912031, 0.0037473, -0.0033693], 
               [-0.0019327, 0.0037024, -0.0016431, -0.9904943], 
               [0.0031495, 0.003414, 0.99052, -0.0008703]])

Gy = np.array([[1, 0, 0, 0], 
               [-0.0018068, 0.0057858, 0.0001531, 0.991508], 
               [-4.8e-5, 9.79e-5, 0.9924849, -0.0001799], 
               [0.0050456, -0.9916214, 0.0002356, 0.0007144]])

Gx2 = np.matmul(Gx, Gx)
Gx3 = np.matmul(Gx2, Gx)

In [ ]:
def getLMatrix(state):
    """
    Calculate the L Matrix from noisy density operator
    """
    a = state[0][0]
    b = state[0][1]
    c = state[1][0]
    d = state[1][1]
    
    x = b+c
    y = (b-c)*1j
    z = 2*a - 1
    
    return 0.5*np.array([[1+z, x-y*1j], 
                         [x+y*1j, 1-z]])

### Get GST Vector

In the GST language, one should use the GST vector of a noisy state to perform calculation. In this case, we have
\begin{equation}
|\rho_{\lambda}\rangle\rangle = \frac{1}{2}\begin{pmatrix}
a+d \\
b+c \\
(b-c)i \\
a-d \\
\end{pmatrix}
\end{equation}

In [9]:
def getGSTVector(state):
    a = state[0][0]
    b = state[0][1]
    c = state[1][0]
    d = state[1][1]
    return 0.5*np.array([a+d, b+c, (b-c)*1j, a-d])

In [10]:
def getDenOper(GSTVector):
    a = GSTVector[0]
    b = GSTVector[1]
    c = GSTVector[2]
    d = GSTVector[3]
    return np.array([[a+d, b-c*1j], 
                     [b+c*1j, a-d]])

In [11]:
getDenOper(getGSTVector(rho0))

array([[ 0.9945645+0.j, -0.0085467+0.j],
       [-0.0269976+0.j,  0.0054355+0.j]])

In [13]:
rho0_GST = getGSTVector(rho0)
rho1_GST = np.matmul(Gx2, rho0_GST)
rhop_GST = np.matmul(Gy, rho0_GST)
rhoi_GST = np.matmul(Gx3, rho0_GST)

rho1 = getDenOper(rho1_GST)
rhop = getDenOper(rhop_GST)
rhoi = getDenOper(rhoi_GST)

### Gamma Matrix Estimation

In [11]:
np.zeros(4)

array([0., 0., 0., 0.])

In [ ]:
gamma1 = np.array([[0.9981, 0.0063], [0.0019, 0.9937]])
gamma2 = np.array([[0.9981, 0.0063], [0.0019, 0.9937]])
gamma3 = np.array([[0.9977, 0.0108], [0.0023, 0.9892]])
gamma4 = np.array([[0.9986, 0.0125], [0.0014, 0.9875]])

In [ ]:
tmp1 = np.kron(gamma1, gamma2)
tmp2 = np.kron(gamma3, gamma4)
Gamma = np.kron(tmp1, tmp2)

In [ ]:
Gamma